# Example Grids

The `example_grid` function automatically generates grids representing some common physical scenarios for testing
    and illustration. This notebook illustrates these examples.

In [ ]:
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.plasma import grids

## axially_magnetized_cylinder

An electric field created by a sphere of potential of radius L/2 with a radial Gaussian distribution:
        
\begin{align}
\rho &= \sqrt{x^2 + y^2}\\
a &= L/4
\end{align}

\begin{equation}
\vec B = 
\begin{cases}
100 \text{ T } \hat z & \rho < a \\
0 \text{ T } & \rho \geq a 
\end{cases}
\end{equation}


In [ ]:
grid = grids.example_grid("axially_magnetized_cylinder", L=1 * u.cm, num=100)
xaxis,yaxis,zaxis = grid.ax0.to(u.mm).value, grid.ax1.to(u.mm).value, grid.ax2.to(u.mm).value
y0 = np.argmin(np.abs(zaxis))
z0 = np.argmin(np.abs(zaxis))

print(grid)

fig, ax = plt.subplots(ncols=2)
fig.subplots_adjust(wspace=0.5)

ax[0].set_aspect("equal")
ax[0].set_xlabel("X (mm)")
ax[0].set_ylabel("Y (mm)")
ax[0].pcolormesh(xaxis, yaxis, grid['B_z'][:,:,z0].T)
ax[0].set_title("B_z (T)")
    
ax[1].set_aspect("equal")
ax[1].set_xlabel("X (mm)")
ax[1].set_ylabel("Z (mm)")
ax[1].pcolormesh(xaxis, zaxis, grid['B_z'][:,y0,:].T)
ax[1].set_title("B_z (T)")


## electrostatic_discontinuity

A discontinuity in the electric field at z=0 with a radial Gaussian profile in the xy plane.
        
\begin{align}
a =& L/2\\
\delta =& a/120\\
\rho =& \sqrt{x^2 + y^2}
\end{align}

\begin{equation}
\phi = [1 - \Gamma(z/\delta)] e^{-(\rho/a)^2}
\end{equation}

\begin{equation}
\vec E = - \nabla \phi
\end{equation}

Where $\Gamma$ is the error function.

In [ ]:
grid = grids.example_grid("electrostatic_discontinuity", L=1 * u.cm, num=100)
xaxis,yaxis,zaxis = grid.ax0.to(u.mm).value, grid.ax1.to(u.mm).value, grid.ax2.to(u.mm).value
x0 = np.argmin(np.abs(xaxis))
y0 = np.argmin(np.abs(yaxis))
z0 = np.argmin(np.abs(zaxis))
zm1 = np.argmin(np.abs(zaxis+1))
Ez = grid['E_z']
Ezmax = np.max(np.abs(Ez))
Et = np.sqrt(grid['E_x']**2 + grid['E_y']**2)
Etmax = np.max(np.abs(Et))

fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(12,4))
fig.subplots_adjust(hspace=1)


ax[0][0].plot(zaxis, grid['phi'][x0,y0,:])
ax[0][0].set_xlabel("Z (mm)")
ax[0][0].set_ylabel("Potential (V)")
ax[0][0].set_title("x=y=0")

ax[0][1].set_aspect("equal")
ax[0][1].set_xlabel("X (mm)")
ax[0][1].set_ylabel("Z (mm)")
ax[0][1].pcolormesh(xaxis, zaxis, grid['phi'][:,y0,:].T)
ax[0][1].set_title("Potential (V)\nat y=0")

ax[0][2].set_aspect("equal")
ax[0][2].set_xlabel("X (mm)")
ax[0][2].set_ylabel("Y (mm)")
ax[0][2].pcolormesh(xaxis, yaxis, grid['phi'][:,:,zm1].T)
ax[0][2].set_title("Potential (V)\nat z=-1")

ax[1][0].plot(zaxis, Ez[x0,y0,:])
ax[1][0].set_xlabel("Z (mm)")
ax[1][0].set_ylabel("E_Z (V/m)")
ax[1][0].set_title("x=y=0")

ax[1][1].set_aspect("equal")
ax[1][1].set_xlabel("X (mm)")
ax[1][1].set_ylabel("Z (mm)")
ax[1][1].pcolormesh(xaxis, zaxis, Ez[:,y0,:].T,
                    vmin=-Ezmax, vmax=Ezmax, cmap='seismic')
ax[1][1].set_title("E_z (V/m)\nat y=0")

ax[1][2].set_aspect("equal")
ax[1][2].set_xlabel("X (mm)")
ax[1][2].set_ylabel("Y (mm)")
ax[1][2].pcolormesh(xaxis, yaxis, Et[:,:,zm1].T)
ax[1][2].set_title("sqrt(Ex^2 + Ey^2) (V/m)\nat z=0")
    
    

print(grid)


## electrostatic_gaussian_sphere

An electric field created by a sphere
        of potential of radius L/2 with a radial Gaussian distribution:
        
\begin{align}
\rho =& \sqrt{x^2 + y^2}\\
a =& L/3\\
b =& L/2
\end{align}

\begin{equation}
\phi = e^{-(r/a)^2}
\end{equation}

\begin{equation}
\vec E =
\begin{cases}
0 & r \geq b \\
-\nabla \phi & r < b
\end{cases}
\end{equation}


In [ ]:
grid = grids.example_grid("electrostatic_gaussian_sphere", L=1 * u.cm, num=100)
xaxis,yaxis,zaxis = grid.ax0.to(u.mm).value, grid.ax1.to(u.mm).value, grid.ax2.to(u.mm).value
z0 = np.argmin(np.abs(zaxis))

print(grid)

fig, ax = plt.subplots(ncols=3, sharey='all', figsize=(12,4))
fig.subplots_adjust(wspace=0.5)

plots = {'phi':'Electric potential (V)', 'E_x':'Ex (V/m)', 'E_y':'Ey (V/m)'}

ax[0].set_ylabel('Y (mm)')
for i,key in enumerate(plots.keys()):
    ax[i].set_xlabel('X (mm)')
    ax[i].set_aspect('equal')
    ax[i].set_title(plots[key])
    ax[i].pcolormesh(xaxis, yaxis, grid[key][:,:,z0])